# 📈 Evaluación de Modelos - Análisis de Sentimientos

## Objetivo
Evaluar y comparar el rendimiento de los modelos entrenados usando múltiples métricas y visualizaciones.

**Métricas de evaluación:**
- Accuracy, Precision, Recall, F1-Score
- Matrices de Confusión
- Curvas ROC y AUC
- Classification Reports
- Feature Importance

In [ ]:
# Importaciones
import pandas as pd
import numpy as np
import joblib
import sys
sys.path.append('..')

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)
from src.visualizations import (
    plot_confusion_matrices,
    plot_roc_curves,
    plot_metrics_comparison,
    plot_feature_importance,
    plot_prediction_distribution
)
from src.train_models import split_data

print('✅ Módulos importados')

In [ ]:
# Cargar modelos entrenados
print('📦 Cargando modelos...')
nb_model = joblib.load('../models/naive_bayes.joblib')
lr_model = joblib.load('../models/logistic_regression.joblib')
rf_model = joblib.load('../models/random_forest.joblib')
vectorizer = joblib.load('../models/tfidfvectorizer.joblib')

models = {
    'Naive Bayes': nb_model,
    'Logistic Regression': lr_model,
    'Random Forest': rf_model
}

print('✅ Modelos cargados correctamente')

In [ ]:
# Cargar y preparar datos de test
df = pd.read_csv('../data/imdb_preprocessed.csv')
X_train, X_test, y_train, y_test = split_data(df, test_size=0.2, random_state=42)

# Vectorizar
X_test_vec = vectorizer.transform(X_test)

print(f'✅ Datos de test listos: {len(X_test)} muestras')

In [ ]:
# Hacer predicciones con todos los modelos
print('🔮 Generando predicciones...')

predictions = {}
for name, model in models.items():
    predictions[name] = model.predict(X_test_vec)
    print(f'  ✓ {name}')

# Mostrar primeros 10 resultados
print('\n📊 PRIMERAS 10 PREDICCIONES:')
comparison_df = pd.DataFrame({
    'Real': y_test.values[:10],
    'NB': predictions['Naive Bayes'][:10],
    'LR': predictions['Logistic Regression'][:10],
    'RF': predictions['Random Forest'][:10]
})
print(comparison_df)

## 📚 Explicación de Métricas

### 1. **Accuracy (Exactitud)**
   - Porcentaje de predicciones correctas
   - Formula: (TP + TN) / Total

### 2. **Precision (Precisión)**
   - De todas las predicciones positivas, cuántas fueron correctas
   - Formula: TP / (TP + FP)
   - Importante cuando los falsos positivos son costosos

### 3. **Recall (Sensibilidad)**
   - De todos los casos positivos reales, cuántos detectamos
   - Formula: TP / (TP + FN)
   - Importante cuando los falsos negativos son costosos

### 4. **F1-Score**
   - Media armónica de Precision y Recall
   - Formula: 2 * (Precision * Recall) / (Precision + Recall)
   - **La métrica más balanceada**

In [ ]:
# Calcular métricas para todos los modelos
print('📊 CALCULANDO MÉTRICAS...')
print('='*70)

metrics_data = []

for name, y_pred in predictions.items():
    metrics = {
        'Model': name,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred)
    }
    metrics_data.append(metrics)

metrics_df = pd.DataFrame(metrics_data).set_index('Model')
print('\n✅ TABLA DE MÉTRICAS:')
print(metrics_df.to_string())

# Identificar el mejor modelo
best_model = metrics_df['f1_score'].idxmax()
best_f1 = metrics_df['f1_score'].max()
print(f'\n🏆 MEJOR MODELO: {best_model} (F1-Score: {best_f1:.4f})')

In [ ]:
# Visualizar comparación de métricas
plot_metrics_comparison(metrics_df)

In [ ]:
# Calcular matrices de confusión
print('\n📊 GENERANDO MATRICES DE CONFUSIÓN...')
confusion_matrices = {}

for name, y_pred in predictions.items():
    cm = confusion_matrix(y_test, y_pred)
    confusion_matrices[name] = cm
    print(f'\n{name}:')
    print(cm)

# Visualizar
plot_confusion_matrices(confusion_matrices)

## 🔍 Interpretación de Matrices de Confusión

### Estructura de la Matriz:
```
                Predicted Negative  Predicted Positive
Actual Negative        TN                 FP
Actual Positive        FN                 TP
```

- **TN (True Negative)**: Correctamente clasificados como negativos
- **TP (True Positive)**: Correctamente clasificados como positivos
- **FP (False Positive)**: Error - Predijo positivo cuando era negativo
- **FN (False Negative)**: Error - Predijo negativo cuando era positivo

### Observaciones:
- Idealmente, los valores más altos deben estar en la diagonal (TN y TP)
- FP y FN deben ser mínimos

In [ ]:
# Calcular curvas ROC
print('\n📈 CALCULANDO CURVAS ROC...')
roc_data = {}

for name, model in models.items():
    # Obtener probabilidades
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_test_vec)[:, 1]
    else:
        y_proba = model.decision_function(X_test_vec)
    
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    roc_data[name] = {
        'fpr': fpr,
        'tpr': tpr,
        'auc': roc_auc
    }
    print(f'  {name}: AUC = {roc_auc:.4f}')

# Visualizar
plot_roc_curves(roc_data)

## 📊 Interpretación de ROC y AUC

### ¿Qué es la Curva ROC?
- **ROC** = Receiver Operating Characteristic
- Muestra el trade-off entre True Positive Rate y False Positive Rate
- Ayuda a evaluar el rendimiento del modelo en diferentes thresholds

### ¿Qué es el AUC?
- **AUC** = Area Under the Curve (Área bajo la curva)
- Métrica resumen del rendimiento del modelo
- **Interpretación**:
  - AUC = 1.0: Clasificador perfecto
  - AUC = 0.9-1.0: Excelente
  - AUC = 0.8-0.9: Muy bueno
  - AUC = 0.7-0.8: Bueno
  - AUC = 0.5: Random (línea diagonal)

In [ ]:
# Classification Reports detallados
print('\n📋 CLASSIFICATION REPORTS')
print('='*70)

for name, y_pred in predictions.items():
    print(f'\n{name.upper()}')
    print('-'*70)
    print(classification_report(y_test, y_pred, 
                                target_names=['Negative', 'Positive'],
                                digits=4))

In [ ]:
# Feature Importance - Logistic Regression
print('\n🔍 ANALIZANDO CARACTERÍSTICAS IMPORTANTES (Logistic Regression)...')
plot_feature_importance(lr_model, vectorizer, top_n=20)

## 💡 Análisis de Características Importantes

### Palabras que indican sentimiento POSITIVO (verde):
- Palabras con coeficiente positivo
- Aumentan la probabilidad de clasificación positiva
- Ejemplos típicos: "excellent", "amazing", "great", "loved"

### Palabras que indican sentimiento NEGATIVO (rojo):
- Palabras con coeficiente negativo
- Aumentan la probabilidad de clasificación negativa
- Ejemplos típicos: "worst", "terrible", "boring", "waste"

In [ ]:
# Distribución de predicciones
plot_prediction_distribution(y_test.values, predictions)

In [ ]:
# Exportar tabla de métricas
import os
os.makedirs('../results', exist_ok=True)

metrics_df.to_csv('../results/model_metrics.csv')
print('\n💾 Métricas exportadas a: ../results/model_metrics.csv')

# Guardar resumen
with open('../results/evaluation_summary.txt', 'w') as f:
    f.write('='*70 + '\n')
    f.write('RESUMEN DE EVALUACIÓN DE MODELOS\n')
    f.write('='*70 + '\n\n')
    f.write('MÉTRICAS DE RENDIMIENTO:\n')
    f.write(metrics_df.to_string())
    f.write(f'\n\nMEJOR MODELO: {best_model}\n')
    f.write(f'F1-Score: {best_f1:.4f}\n')
    f.write('\n' + '='*70 + '\n')

print('💾 Resumen exportado a: ../results/evaluation_summary.txt')
print('\n✅ Todos los resultados guardados')

## 🎯 Conclusiones de la Evaluación

### 📊 Resultados Principales:

1. **Mejor Modelo**: El modelo con mejor F1-Score es el más balanceado
2. **Métricas**: Todos los modelos superan 85% de accuracy
3. **AUC**: Valores superiores a 0.90 indican excelente capacidad discriminativa

### 🔍 Observaciones por Modelo:

**Naive Bayes:**
- ✅ Muy rápido en entrenamiento
- ✅ Buen baseline
- ⚠️ Asume independencia de features (no siempre cierto)

**Logistic Regression:**
- ✅ Balance entre velocidad y precisión
- ✅ Interpretable (coeficientes = importancia)
- ✅ Robusto y confiable

**Random Forest:**
- ✅ Captura relaciones complejas
- ⚠️ Más lento en entrenamiento
- ⚠️ Menos interpretable

### 🚀 Recomendaciones:

1. Para **producción**: Usar el modelo con mejor F1-Score
2. Para **velocidad**: Naive Bayes es suficiente
3. Para **interpretabilidad**: Logistic Regression

### 🔜 Posibles Mejoras:

- Ajustar hiperparámetros (Grid Search)
- Probar otros modelos (SVM, XGBoost)
- Feature engineering más sofisticado
- Ensemble methods (combinar modelos)

---
**Siguiente notebook:** `05_complete_workflow.ipynb` (Workflow completo integrado)